# Lending Bad Debt Notebook

This notebook searches through the lending markets on Arbitrum and Ethereum for bad debt in each lending market.

It finds all the markets by looking at all the markets deployed from the OneWayLendingContracts on each network

You simply have to input Ethereum and Arbitrum RPCs below to have the notebook function correctly.

The print flags just enable or disable printing or different data pieces, you can print each user with bad debt, print which markets have no bad debt, and print the total bad debt of each market.  These are all enabled by default, switch them to `False` if you would like to disable them

In [1]:
# Input your Ethereum and Arbitrum RPCs below
# easiest way to get an RPC URL is to get a free account at Alchemy.com
ETHEREUM_HTTP_RPC = "https://eth-mainnet.g.alchemy.com/v2/BrfnLS7UL-EyH1stExHDMMIYQUPGDomJ"
ARBITRUM_HTTP_RPC = "https://arb-mainnet.g.alchemy.com/v2/vZDjkn0-x8LHv_HxWG0hjOXopNOuBeur"

# PRINT FLAGS
# print each individual user with bad debt
PRINT_BD_USERS = True
# print which markets have no bad debt
PRINT_GOOD_MARKETS = True
# print bad debt totals
PRINT_BD_TOTALS = True

In [2]:
ETHEREUM_ONEWAYLENDINGFACTORY_ADDR = '0xeA6876DDE9e3467564acBeE1Ed5bac88783205E0'
ARBITRUM_ONEWAYLENDINGFACTORY_ADDR = '0xcaEC110C784c9DF37240a8Ce096D352A75922DeA'

NETWORKS = ['ethereum', 'arbitrum']

In [3]:
# uncomment the following lines to install the web3 library within a notebook environment
# %%capture
# !pip install web3
from web3 import Web3

def read_abi(filename):
    with open(filename, 'r') as file:
        return file.read()

def get_w3_and_factory(network):
    if network == 'ethereum':
        w3 = Web3(Web3.HTTPProvider(ETHEREUM_HTTP_RPC))
        factory = w3.eth.contract(address=ETHEREUM_ONEWAYLENDINGFACTORY_ADDR, abi=read_abi('abis/OneWayLendingFactory.abi'))
    elif network == 'arbitrum':
        w3 = Web3(Web3.HTTPProvider(ARBITRUM_HTTP_RPC))
        factory = w3.eth.contract(address=ARBITRUM_ONEWAYLENDINGFACTORY_ADDR, abi=read_abi('abis/OneWayLendingFactory.abi'))
    
    if w3.is_connected():
        print(f'\nSuccessfully connected to {network}.')
        
        return w3, factory
    else:
        raise Exception("Failed to connect to web3.")

In [4]:
# read in ABIs for required contracts
controllerAbi = read_abi('abis/controller.abi')
llammaAbi = read_abi('abis/LLAMMA.abi')
erc20Abi = read_abi('abis/ERC20.abi')

In [5]:
from datetime import date

today_date = date.today().strftime("%Y-%m-%d")
print(f'BAD DEBT DATA ACCURATE AS OF {today_date}')

# search through networks to find bad debt
for network in NETWORKS:
    w3, factory = get_w3_and_factory(network)

    i = 0
    while(True):
        # no way to find how many markets there are, so continue calling until end of list
        try:
            controllerAddr = factory.functions.controllers(i).call()
        except:
            break
        controller = w3.eth.contract(address=controllerAddr, abi=controllerAbi)

        i += 1

        # some markets aren't properly initialize, so skip them
        try:
            llammaAddr = controller.functions.amm().call()
            llamma = w3.eth.contract(address=llammaAddr, abi=llammaAbi)
            price = llamma.functions.price_oracle().call()
        except:
            print(f'Error finding market data - Network: {network}, Market Address: {controllerAddr}')
            continue

        # get collateral and debt token info
        collateralTokenContract = w3.eth.contract(address=controller.functions.collateral_token().call(), abi=erc20Abi)
        collateralToken = collateralTokenContract.functions.symbol().call()
        collateralDecimals = collateralTokenContract.functions.decimals().call()
        debtTokenContract = w3.eth.contract(address=controller.functions.borrowed_token().call(), abi=erc20Abi)
        debtToken = debtTokenContract.functions.symbol().call()
        debtDecimals = debtTokenContract.functions.decimals().call()

        # find if there are any users that can be liquidated.  If there are, check them for bad debt
        # usersToLiq is a list of users with loans that can be liquidated.  If the length is 0, there are no users to liquidate
        usersToLiq = controller.functions.users_to_liquidate().call()
        badDebtFlag = False
        totalBadDebt = 0
        if len(usersToLiq) > 0:
            for user in usersToLiq:
                # each user is in the form [userAddr, stablecoinAmount, collateralAmount, debt, health]
                collateral = user[2]/10**collateralDecimals
                swappedCollateral = user[1]/10**debtDecimals
                collateralTotal = round(collateral*(price/10**18) + swappedCollateral, 2)
                debt = round(user[3]/10**debtDecimals, 2)
                badDebt = round(debt - collateralTotal, 2)
                if badDebt > 0:
                    if (not badDebtFlag):
                        badDebtFlag = True
                        print('')
                    if (PRINT_BD_USERS):
                        print(f'Bad Debt Found - Amount {badDebt}, Network: {network}, Market: {collateralToken}/{debtToken} - {controllerAddr}, \n     .... User: {user[0]}, Debt: {debt}, \n     .... Collateral Total: {collateralTotal}, Collateral {collateralToken}: {round(collateral,2)}, Collateral {debtToken}: {round(swappedCollateral,2)}')
                    totalBadDebt += badDebt
        if not badDebtFlag:
            if (PRINT_GOOD_MARKETS):
                print(f'No Bad Debt - Network: {network}, Market: {collateralToken}/{debtToken} - {controllerAddr}')
        else:
            if (PRINT_BD_TOTALS):
                print(f'Total Bad Debt {round(totalBadDebt, 2)} - Network: {network}, Market: {collateralToken}/{debtToken} - {controllerAddr}\n')

BAD DEBT DATA ACCURATE AS OF 2024-05-30

Successfully connected to ethereum.
No Bad Debt - Network: ethereum, Market: wstETH/crvUSD - 0x1E0165DbD2019441aB7927C018701f3138114D71
No Bad Debt - Network: ethereum, Market: WETH/crvUSD - 0xaade9230AA9161880E13a38C83400d3D1995267b
No Bad Debt - Network: ethereum, Market: tBTC/crvUSD - 0x413FD2511BAD510947a91f5c6c79EBD8138C29Fc
No Bad Debt - Network: ethereum, Market: CRV/crvUSD - 0xEdA215b7666936DEd834f76f3fBC6F323295110A
No Bad Debt - Network: ethereum, Market: crvUSD/CRV - 0xC510d73Ad34BeDECa8978B6914461aA7b50CF3Fc
No Bad Debt - Network: ethereum, Market: crvUSD/WETH - 0xa5D9137d2A1Ee912469d911A8E74B6c77503bac8
No Bad Debt - Network: ethereum, Market: crvUSD/tBTC - 0xe438658874b0acf4D81c24172E137F0eE00621b8
No Bad Debt - Network: ethereum, Market: sUSDe/crvUSD - 0x98Fc283d6636f6DCFf5a817A00Ac69A3ADd96907
No Bad Debt - Network: ethereum, Market: UwU/crvUSD - 0x09dBDEB3b301A4753589Ac6dF8A178C7716ce16B
No Bad Debt - Network: ethereum, Market: 